![alt](https://research.utm.my/wp-content/uploads/sites/26/2022/06/logo-300x122.png)
# Center for Artificial Intelligence and Robotics
#### Universiti Teknologi Malaysia


### Detection Inference - FasterRCNN

*Author: Dr. Ibrahim, Azzam, Thaqif & Syahmi*

**FasterRCNN for object detection.**

_ | _

In [ ]:
%%bash
pip install -U matplotlib

In [ ]:
import torch
import torchvision.transforms as transforms
from torchvision.models.detection import fasterrcnn_resnet50_fpn
from PIL import Image
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import requests

# URL to the COCO class names
url = 'https://raw.githubusercontent.com/amikelive/coco-labels/master/coco-labels-paper.txt'

# Download the class names
response = requests.get(url)
coco_classes = ['background'] + response.text.strip().split('\n')

# Load the Faster R-CNN model pre-trained on COCO dataset
model = fasterrcnn_resnet50_fpn(pretrained=True)
model.eval()

# Load an image
image_path = 'dog.jpg'
image = Image.open(image_path).convert('RGB')
# Preprocess the image
transform = transforms.Compose([
    transforms.ToTensor(),
])
input_tensor = transform(image).unsqueeze(0)

# Perform object detection
with torch.no_grad():
    predictions = model(input_tensor)

# Process the predictions
pred_boxes = predictions[0]['boxes'].cpu().numpy()
pred_scores = predictions[0]['scores'].cpu().numpy()
pred_labels = predictions[0]['labels'].cpu().numpy()

# Set a confidence threshold
confidence_threshold = 0.5
filtered_boxes = pred_boxes[pred_scores >= confidence_threshold]
filtered_labels = pred_labels[pred_scores >= confidence_threshold]

# Map the labels to class names
pred_class_names = [coco_classes[label] for label in filtered_labels]

# Visualize the results
fig, ax = plt.subplots(1)
ax.imshow(image)

# Add bounding boxes to the image
for box, label in zip(filtered_boxes, pred_class_names):
    x1, y1, x2, y2 = box
    rect = patches.Rectangle((x1, y1), x2 - x1, y2 - y1, linewidth=2, edgecolor='r', facecolor='none')
    ax.add_patch(rect)
    plt.text(x1, y1, f'{label}', bbox=dict(facecolor='white', alpha=0.5))

plt.show()